In [ ]:
from naive_rag import naive_rag
from init_vectorstore import init_vectorstore
from langchain_experimental.text_splitter import SemanticChunker
from ragas_func import ragas_with_params
import os
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
os.environ["OPENAI_API_KEY"] = "sk-proj-eZ7RgSAYfMCqsHtKkt7DT3BlbkFJ2WvLlx4KoAai5PUJrodw"

## Vectorstores: ChromaDB, FAISS, Qdrant
## Embeddings: "text-embedding-3-large", "paraphrase-multilingual-mpnet-base-v2", "multilingual-e5-large"
## Text splitters: CharacterTextSplitter, RecursiveCharacterTextSplitter, SemanticChunker 


In [ ]:
instances = 20
file_path = "/Users/adrianfolge/Documents/lokal:skole/Master/data/synthetic_data/vol2_questions_and_answers_ytterligere_revidert.csv"

In [ ]:
references = load_dataset('csv', data_files=file_path, split=f"train[:{instances}]")
refs = references["svar"]
questions = references["spørsmål"]

In [ ]:
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
character_text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
semantic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="percentile"
)

In [ ]:
embeddings_para = SentenceTransformerEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings_multiling = SentenceTransformerEmbeddings(model_name='intfloat/multilingual-e5-large')
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
text_splitters = [semantic_text_splitter,recursive_text_splitter, character_text_splitter]
embeddings = [embeddings_para, embeddings_multiling, embeddings_openai]

In [ ]:
list_of_scores = []
for text_splitter in text_splitters:
    for embedding in embeddings:
        databases = init_vectorstore(embedding, text_splitter)
        naive_rag_list = naive_rag(instances, file_path, databases)
        naive_rag_score = ragas_with_params(naive_rag_list, questions, naive_rag_list, refs)
        score = {
            "text_splitter": text_splitter,
            "embedding model": embedding,
            "score": naive_rag_score
        }
        list_of_scores.append(score)
        

In [ ]:
with open('/Users/adrianfolge/Documents/lokal:skole/Master/master_folder/eval_data/embeddings_and_text_splitters_Qdrant.txt', 'w') as f:
    for item in list_of_scores:
        f.write(str(item) + '\n')

In [ ]:
# Data
data = {
    'Vectorstore': ['FAISS', 'FAISS', 'FAISS', 'FAISS', 'FAISS', 'FAISS', 'FAISS', 'FAISS', 'FAISS'],
    'Embedding model': ['mpnet', 'e5', 'Open AI', 'mpnet', 'e5', 'Open AI', 'mpnet', 'e5', 'Open AI'],
    'Text splitter': ['Semantic', 'Semantic', 'Semantic', 'Recursive', 'Recursive', 'Recursive', 'Character', 'Character', 'Character'],
    'Context precision': [0.7000, 0.8000, 0.8500, 0.9500, 0.9500, 0.9500, 0.9000, 0.9000, 0.9500],
    'Faithfulness': [0.9531, 0.9667, 0.9750, 0.9692, 0.9900, 0.9692, 0.9608, 0.9775, 0.9817],
    'Answer relevancy': [0.7933, 0.7846, 0.8153, 0.8551, 0.8294, 0.8373, 0.8676, 0.8168, 0.8395],
    'Context recall': [0.8875, 0.9250, 0.9375, 0.8625, 0.8750, 0.8375, 0.8875, 0.9000, 0.8625]
}

# Create DataFrame
df = pd.DataFrame(data)

# Calculate average score for each row
df['Average'] = df[['Context precision', 'Faithfulness', 'Answer relevancy', 'Context recall']].mean(axis=1)

# Plot
plt.figure(figsize=(10, 6))
plt.bar(df.index, df['Average'], color='skyblue')
plt.ylabel('Average Score')
plt.title('Average Score for Qdrant DB (Norwegian)')
plt.xticks(df.index, df['Embedding model'] + ' - ' + df['Text splitter'], rotation=45)
plt.ylim(0.8, 1)
plt.tight_layout()
plt.show()